# Example 2: Intermediate Document Summary with Human Editing

## Problem Statement
Create a LangGraph that processes a document to generate a summary, allows a human to edit the summary for accuracy and completeness, and then uses the human-edited summary for a follow-up task (like generating action items or creating a presentation outline). This demonstrates more sophisticated human-in-the-loop interaction where human input directly improves the AI's work product.

## Features
- Document processing and summarization
- Human editing interface for summary refinement
- Follow-up task generation using edited summary
- Version tracking and comparison
- Error handling and input validation

In [ ]:
# Install required packages (run this cell if packages are not installed)
# !pip install langchain langgraph langchain-openai pytest difflib

In [ ]:
import os
from typing import TypedDict, Annotated, List, Dict, Optional
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
import json
from datetime import datetime
import difflib
import re

# Set up OpenAI API key (make sure this is set in your environment)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("📚 Document Summary Editing with Human-in-the-Loop")
print("=" * 55)

## Step 1: Define the State
The state tracks the document, AI-generated summary, human-edited version, and follow-up tasks.

In [ ]:
class DocumentSummaryState(TypedDict):
    messages: Annotated[List, add_messages]
    original_document: str
    ai_summary: str
    human_edited_summary: str
    summary_approved: bool
    editing_notes: str
    follow_up_task: str
    follow_up_result: str
    version_history: List[Dict]
    processing_stage: str

print("✅ State defined with fields:")
print("   - messages: conversation history")
print("   - original_document: source document text")
print("   - ai_summary: AI-generated summary")
print("   - human_edited_summary: human-refined summary")
print("   - summary_approved: human approval status")
print("   - editing_notes: human feedback and notes")
print("   - follow_up_task: task to perform with edited summary")
print("   - follow_up_result: result of follow-up task")
print("   - version_history: track summary versions")
print("   - processing_stage: current workflow stage")

## Step 2: Helper Functions
Utility functions for document processing and comparison.

In [ ]:
def create_version_entry(summary: str, editor: str, notes: str = "") -> Dict:
    """Create a version history entry."""
    return {
        "timestamp": datetime.now().isoformat(),
        "summary": summary,
        "editor": editor,
        "notes": notes,
        "word_count": len(summary.split())
    }

def show_diff(original: str, edited: str) -> str:
    """Show differences between original and edited text."""
    diff = list(difflib.unified_diff(
        original.splitlines(keepends=True),
        edited.splitlines(keepends=True),
        fromfile='AI Summary',
        tofile='Human Edited',
        lineterm=''
    ))
    return ''.join(diff) if diff else "No changes detected."

def validate_document(document: str) -> tuple[bool, str]:
    """Validate document format and content."""
    if not document or not document.strip():
        return False, "Document is empty"
    
    word_count = len(document.split())
    if word_count < 10:
        return False, f"Document too short ({word_count} words). Need at least 10 words."
    
    if word_count > 5000:
        return False, f"Document too long ({word_count} words). Maximum 5000 words."
    
    return True, f"Document validated ({word_count} words)"

print("✅ Helper functions defined: create_version_entry, show_diff, validate_document")

## Step 3: Define the Graph Nodes
Each node represents a step in the document processing and editing workflow.

In [ ]:
def process_document(state: DocumentSummaryState) -> DocumentSummaryState:
    """Process and validate the input document."""
    print("📄 Processing document...")
    
    try:
        messages = state["messages"]
        if not messages:
            raise ValueError("No messages found")
        
        # Extract document from the first message
        document = messages[0].content
        
        # Validate document
        is_valid, validation_msg = validate_document(document)
        print(f"📋 Validation: {validation_msg}")
        
        if not is_valid:
            raise ValueError(validation_msg)
        
        return {
            "original_document": document,
            "processing_stage": "document_processed",
            "version_history": []
        }
        
    except Exception as e:
        print(f"❌ Error processing document: {e}")
        return {
            "original_document": f"Error: {str(e)}",
            "processing_stage": "error"
        }


def generate_ai_summary(state: DocumentSummaryState) -> DocumentSummaryState:
    """Generate AI summary of the document."""
    print("🤖 Generating AI summary...")
    
    try:
        document = state["original_document"]
        
        if document.startswith("Error:"):
            return {"ai_summary": "Cannot summarize due to document processing error."}
        
        # Create summarization prompt
        system_msg = SystemMessage(content="""You are an expert document summarizer. 
        Create a clear, concise summary that captures the key points, main arguments, 
        and important details. Structure your summary with bullet points or paragraphs 
        as appropriate. Aim for 150-300 words depending on document length.""")
        
        user_msg = HumanMessage(content=f"Please summarize this document:\n\n{document}")
        
        llm = ChatOpenAI(model="gpt-4", temperature=0.3)
        response = llm.invoke([system_msg, user_msg])
        
        ai_summary = response.content
        print(f"📝 AI summary generated ({len(ai_summary.split())} words)")
        
        # Create version history entry
        version_entry = create_version_entry(
            ai_summary, 
            "AI", 
            "Initial AI-generated summary"
        )
        
        return {
            "ai_summary": ai_summary,
            "processing_stage": "summary_generated",
            "version_history": state.get("version_history", []) + [version_entry],
            "messages": [response]
        }
        
    except Exception as e:
        print(f"❌ Error generating summary: {e}")
        return {
            "ai_summary": f"Error generating summary: {str(e)}",
            "processing_stage": "error"
        }


def request_human_editing(state: DocumentSummaryState) -> DocumentSummaryState:
    """Request human editing of the AI summary."""
    print("👤 Requesting human editing...")
    
    try:
        ai_summary = state["ai_summary"]
        
        if ai_summary.startswith("Error"):
            return {
                "summary_approved": False,
                "editing_notes": "Cannot edit due to summary generation error"
            }
        
        print(f"\n{'='*70}")
        print("📝 SUMMARY EDITING INTERFACE")
        print(f"{'='*70}")
        print("\n🤖 AI-Generated Summary:")
        print("-" * 40)
        print(ai_summary)
        print("-" * 40)
        
        print("\n📋 Options:")
        print("1. Type 'approve' to accept the summary as-is")
        print("2. Type 'edit' to make changes")
        print("3. Type 'regenerate' to ask AI to try again")
        
        while True:
            action = input("\nWhat would you like to do? (approve/edit/regenerate): ").strip().lower()
            
            if action == 'approve':
                print("✅ Summary approved without changes")
                return {
                    "human_edited_summary": ai_summary,
                    "summary_approved": True,
                    "editing_notes": "Approved without changes",
                    "processing_stage": "summary_approved"
                }
            
            elif action == 'edit':
                print("\n✏️ Edit Mode")
                print("Instructions: Copy the summary above and make your changes below.")
                print("Type 'DONE' on a new line when finished.\n")
                
                edited_lines = []
                print("Enter your edited summary:")
                
                while True:
                    line = input()
                    if line.strip().upper() == 'DONE':
                        break
                    edited_lines.append(line)
                
                edited_summary = '\n'.join(edited_lines).strip()
                
                if not edited_summary:
                    print("❌ Empty summary. Please try again.")
                    continue
                
                # Get editing notes
                notes = input("\nOptional: Add notes about your changes: ").strip()
                
                # Show diff
                print("\n🔍 Changes made:")
                diff = show_diff(ai_summary, edited_summary)
                if diff != "No changes detected.":
                    print(diff)
                else:
                    print("No changes detected.")
                
                # Create version entry
                version_entry = create_version_entry(
                    edited_summary,
                    "Human",
                    notes or "Human edited version"
                )
                
                print("✅ Summary edited successfully")
                return {
                    "human_edited_summary": edited_summary,
                    "summary_approved": True,
                    "editing_notes": notes or "Human edited version",
                    "processing_stage": "summary_approved",
                    "version_history": state.get("version_history", []) + [version_entry]
                }
            
            elif action == 'regenerate':
                feedback = input("What should be improved in the summary? ").strip()
                print("🔄 Requesting regeneration with feedback")
                return {
                    "summary_approved": False,
                    "editing_notes": feedback,
                    "processing_stage": "regenerate_requested"
                }
            
            else:
                print("Please enter 'approve', 'edit', or 'regenerate'")
        
    except Exception as e:
        print(f"❌ Error in editing interface: {e}")
        return {
            "summary_approved": False,
            "editing_notes": f"Error in editing: {str(e)}",
            "processing_stage": "error"
        }


def regenerate_summary(state: DocumentSummaryState) -> DocumentSummaryState:
    """Regenerate summary based on human feedback."""
    print("🔄 Regenerating summary with feedback...")
    
    try:
        document = state["original_document"]
        feedback = state["editing_notes"]
        
        system_msg = SystemMessage(content=f"""You are an expert document summarizer. 
        The user has provided feedback on your previous summary. Please create an improved summary 
        that addresses their feedback.
        
        User feedback: {feedback}
        
        Create a clear, concise summary that incorporates this feedback while maintaining 
        professional quality and completeness.""")
        
        user_msg = HumanMessage(content=f"Please summarize this document based on the feedback:\n\n{document}")
        
        llm = ChatOpenAI(model="gpt-4", temperature=0.3)
        response = llm.invoke([system_msg, user_msg])
        
        new_summary = response.content
        print(f"📝 Regenerated summary ({len(new_summary.split())} words)")
        
        # Create version entry
        version_entry = create_version_entry(
            new_summary,
            "AI",
            f"Regenerated based on feedback: {feedback}"
        )
        
        return {
            "ai_summary": new_summary,
            "processing_stage": "summary_generated",
            "editing_notes": "",  # Clear previous feedback
            "version_history": state.get("version_history", []) + [version_entry],
            "messages": state.get("messages", []) + [response]
        }
        
    except Exception as e:
        print(f"❌ Error regenerating summary: {e}")
        return {
            "ai_summary": f"Error regenerating summary: {str(e)}",
            "processing_stage": "error"
        }


def execute_follow_up_task(state: DocumentSummaryState) -> DocumentSummaryState:
    """Execute follow-up task using the approved summary."""
    print("🎯 Executing follow-up task...")
    
    try:
        summary = state["human_edited_summary"]
        task = state.get("follow_up_task", "")
        
        if not task:
            # Ask user what follow-up task to perform
            print("\n📋 Follow-up Task Options:")
            print("1. Generate action items")
            print("2. Create presentation outline")
            print("3. Identify key stakeholders")
            print("4. Generate questions for discussion")
            print("5. Custom task")
            
            while True:
                choice = input("\nSelect follow-up task (1-5): ").strip()
                
                if choice == '1':
                    task = "Generate a list of actionable items based on this summary"
                    break
                elif choice == '2':
                    task = "Create a presentation outline based on this summary"
                    break
                elif choice == '3':
                    task = "Identify key stakeholders mentioned or implied in this summary"
                    break
                elif choice == '4':
                    task = "Generate thoughtful discussion questions based on this summary"
                    break
                elif choice == '5':
                    task = input("Enter your custom task: ").strip()
                    if task:
                        break
                    else:
                        print("Please enter a task description")
                        continue
                else:
                    print("Please enter 1, 2, 3, 4, or 5")
        
        print(f"🎯 Executing task: {task}")
        
        # Execute the follow-up task
        system_msg = SystemMessage(content=f"""You are a helpful assistant. 
        Based on the provided summary, please {task.lower()}. 
        Provide a well-structured, actionable response.""")
        
        user_msg = HumanMessage(content=f"Summary:\n{summary}\n\nTask: {task}")
        
        llm = ChatOpenAI(model="gpt-4", temperature=0.3)
        response = llm.invoke([system_msg, user_msg])
        
        result = response.content
        print(f"✅ Follow-up task completed ({len(result.split())} words)")
        
        return {
            "follow_up_task": task,
            "follow_up_result": result,
            "processing_stage": "completed",
            "messages": state.get("messages", []) + [response]
        }
        
    except Exception as e:
        print(f"❌ Error executing follow-up task: {e}")
        return {
            "follow_up_result": f"Error executing follow-up task: {str(e)}",
            "processing_stage": "error"
        }

print("✅ Nodes defined: process_document, generate_ai_summary, request_human_editing, regenerate_summary, execute_follow_up_task")

## Step 4: Define Routing Logic
The routing logic determines the flow through the graph based on editing decisions and processing status.

In [ ]:
def route_after_editing(state: DocumentSummaryState) -> str:
    """Route based on editing decision."""
    stage = state.get("processing_stage", "")
    approved = state.get("summary_approved", False)
    
    if stage == "error":
        print("🔀 Routing to end due to error")
        return "end"
    elif stage == "regenerate_requested":
        print("🔀 Routing to regeneration")
        return "regenerate"
    elif approved:
        print("🔀 Routing to follow-up task")
        return "follow_up"
    else:
        print("🔀 Routing to editing (retry)")
        return "edit"


def handle_completion(state: DocumentSummaryState) -> DocumentSummaryState:
    """Handle successful completion of the workflow."""
    print("🎉 Workflow completed successfully!")
    
    # Generate completion summary
    version_count = len(state.get("version_history", []))
    final_summary = state.get("human_edited_summary", "")
    follow_up_result = state.get("follow_up_result", "")
    
    completion_msg = f"""Document processing completed successfully!
    
📊 Summary:
- Versions created: {version_count}
- Final summary: {len(final_summary.split())} words
- Follow-up task completed: {len(follow_up_result.split())} words

✅ All tasks completed with human oversight."""
    
    return {
        "messages": [AIMessage(content=completion_msg)],
        "processing_stage": "completed"
    }


def handle_error(state: DocumentSummaryState) -> DocumentSummaryState:
    """Handle errors in the workflow."""
    print("❌ Workflow ended due to error")
    
    error_msg = """Workflow ended due to an error. Please check:
- Document format and content
- API connectivity
- Input validation

You can restart with a new document."""
    
    return {
        "messages": [AIMessage(content=error_msg)]
    }

print("✅ Routing logic defined")

## Step 5: Build the Graph
Construct the LangGraph with all nodes and conditional routing.

In [ ]:
def build_document_summary_graph():
    """Build and compile the document summary editing graph."""
    print("🏗️ Building document summary editing graph...")
    
    workflow = StateGraph(DocumentSummaryState)
    
    # Add nodes
    workflow.add_node("process_doc", process_document)
    workflow.add_node("generate_summary", generate_ai_summary)
    workflow.add_node("human_edit", request_human_editing)
    workflow.add_node("regenerate", regenerate_summary)
    workflow.add_node("follow_up", execute_follow_up_task)
    workflow.add_node("complete", handle_completion)
    workflow.add_node("error", handle_error)
    
    # Set entry point
    workflow.set_entry_point("process_doc")
    
    # Linear flow for initial processing
    workflow.add_edge("process_doc", "generate_summary")
    workflow.add_edge("generate_summary", "human_edit")
    
    # Conditional routing from human editing
    workflow.add_conditional_edges(
        "human_edit",
        route_after_editing,
        {
            "regenerate": "regenerate",
            "follow_up": "follow_up",
            "edit": "human_edit",  # Retry editing
            "end": "error"
        }
    )
    
    # After regeneration, return to editing
    workflow.add_edge("regenerate", "human_edit")
    
    # After follow-up task, complete the workflow
    workflow.add_edge("follow_up", "complete")
    
    # End nodes
    workflow.add_edge("complete", END)
    workflow.add_edge("error", END)
    
    # Compile the graph
    graph = workflow.compile()
    
    print("✅ Graph compiled successfully!")
    print("   Flow: process_doc → generate_summary → human_edit → [regenerate/follow_up] → complete")
    
    return graph

# Build the graph
document_graph = build_document_summary_graph()

## Step 6: Test the Graph
Run the graph with sample documents to demonstrate the editing workflow.

In [ ]:
# Sample documents for testing
SAMPLE_DOCUMENTS = {
    "meeting_notes": """Q3 Strategy Meeting - October 15, 2024

Attendees: Sarah Chen (CEO), Mark Rodriguez (CTO), Lisa Wang (COO), David Kim (CFO)

Key Discussion Points:
1. Product Development Update
   - Mobile app beta testing completed with 95% user satisfaction
   - Desktop version 2.0 on track for December release
   - AI integration features showing promising early results

2. Market Expansion
   - European market entry planned for Q1 2025
   - Partnership discussions with three major distributors ongoing
   - Regulatory compliance review initiated

3. Financial Performance
   - Q3 revenue exceeded targets by 12%
   - Operating costs reduced by 8% through efficiency improvements
   - Cash flow projections remain positive through 2025

4. Team Growth
   - Engineering team expansion: 5 new hires planned for Q4
   - Sales team restructure to focus on enterprise clients
   - Remote work policy updates effective November 1

Action Items:
- Mark to finalize AI integration timeline by Oct 25
- Lisa to complete European market research by Nov 1
- David to prepare Q4 budget proposal by Oct 30
- Sarah to schedule follow-up with board by Nov 15

Next meeting: November 15, 2024""",

    "research_report": """The Impact of Remote Work on Employee Productivity and Well-being

Executive Summary:
This study examines the effects of remote work arrangements on employee productivity and mental well-being across 500 knowledge workers from various industries during 2023-2024.

Methodology:
We conducted surveys, interviews, and performance analysis across companies with different remote work policies: fully remote, hybrid, and traditional office-based.

Key Findings:

Productivity Metrics:
- Remote workers showed 13% higher task completion rates
- 25% reduction in time spent in unproductive meetings
- 18% increase in deep work sessions (2+ hours uninterrupted)
- However, 22% reported difficulty with collaborative tasks

Well-being Indicators:
- 67% reported improved work-life balance
- 31% experienced reduced commute-related stress
- 28% reported feelings of isolation
- 15% struggled with home-office ergonomics

Hybrid Model Results:
- 3-2 split (3 days remote, 2 days office) showed optimal results
- Maintained collaboration benefits while preserving flexibility
- 89% employee satisfaction rate with hybrid arrangement

Recommendations:
1. Implement structured hybrid policies with clear expectations
2. Invest in digital collaboration tools and training
3. Provide ergonomic home office stipends
4. Schedule regular virtual social interactions
5. Establish core collaboration hours for team alignment

Conclusion:
Remote work can enhance productivity and well-being when properly implemented with appropriate support systems and policies."""
}

def run_document_example(document_key: str = "meeting_notes"):
    """Run the document summary editing example."""
    print(f"\n{'='*70}")
    print(f"🚀 Running Document Summary Editing Example")
    print(f"Document: {document_key}")
    print(f"{'='*70}")
    
    if document_key not in SAMPLE_DOCUMENTS:
        print(f"❌ Document '{document_key}' not found. Available: {list(SAMPLE_DOCUMENTS.keys())}")
        return None
    
    document = SAMPLE_DOCUMENTS[document_key]
    
    initial_state = {
        "messages": [HumanMessage(content=document)],
        "original_document": "",
        "ai_summary": "",
        "human_edited_summary": "",
        "summary_approved": False,
        "editing_notes": "",
        "follow_up_task": "",
        "follow_up_result": "",
        "version_history": [],
        "processing_stage": "initial"
    }
    
    try:
        final_state = None
        for state in document_graph.stream(initial_state):
            final_state = state
        
        print("\n" + "="*70)
        print("📋 WORKFLOW RESULTS")
        print("="*70)
        
        if final_state:
            state_values = list(final_state.values())[-1]
            
            # Show version history
            version_history = state_values.get("version_history", [])
            if version_history:
                print(f"\n📚 Version History ({len(version_history)} versions):")
                for i, version in enumerate(version_history, 1):
                    print(f"   {i}. {version['editor']} - {version['word_count']} words - {version['notes']}")
            
            # Show final summary
            final_summary = state_values.get("human_edited_summary", "")
            if final_summary:
                print(f"\n📄 Final Summary:")
                print("-" * 50)
                print(final_summary)
                print("-" * 50)
            
            # Show follow-up result
            follow_up_result = state_values.get("follow_up_result", "")
            follow_up_task = state_values.get("follow_up_task", "")
            if follow_up_result:
                print(f"\n🎯 Follow-up Task Result:")
                print(f"Task: {follow_up_task}")
                print("-" * 50)
                print(follow_up_result)
                print("-" * 50)
        
        print("\n✅ Document processing workflow completed!")
        return final_state

    except Exception as e:
        print(f"❌ Error running example: {e}")
        return None

print("📝 Document examples ready!")
print("\nAvailable documents:")
for key, doc in SAMPLE_DOCUMENTS.items():
    word_count = len(doc.split())
    print(f"   - {key}: {word_count} words")

print("\nTo test the graph, run:")
print('run_document_example("meeting_notes")')
print('# or')
print('run_document_example("research_report")')

In [ ]:
# Run an example (uncomment to test)
# result = run_document_example("meeting_notes")

## Step 7: Unit Tests
Test the core functionality of the document processing workflow.

In [ ]:
import pytest
from unittest.mock import patch, MagicMock

def test_document_validation():
    """Test document validation logic."""
    # Test empty document
    is_valid, msg = validate_document("")
    assert not is_valid
    assert "empty" in msg.lower()
    
    # Test short document
    is_valid, msg = validate_document("Short doc")
    assert not is_valid
    assert "too short" in msg.lower()
    
    # Test valid document
    valid_doc = "This is a valid document with more than ten words to pass validation."
    is_valid, msg = validate_document(valid_doc)
    assert is_valid
    assert "validated" in msg.lower()
    
    print("✅ test_document_validation passed")

def test_version_history():
    """Test version history creation."""
    summary = "This is a test summary"
    version = create_version_entry(summary, "AI", "Initial version")
    
    assert version["summary"] == summary
    assert version["editor"] == "AI"
    assert version["notes"] == "Initial version"
    assert version["word_count"] == 5
    assert "timestamp" in version
    
    print("✅ test_version_history passed")

def test_diff_functionality():
    """Test diff generation between text versions."""
    original = "The quick brown fox jumps over the lazy dog."
    edited = "The quick brown fox leaps over the lazy dog."
    
    diff = show_diff(original, edited)
    assert "jumps" in diff
    assert "leaps" in diff
    
    # Test no changes
    no_diff = show_diff(original, original)
    assert no_diff == "No changes detected."
    
    print("✅ test_diff_functionality passed")

def test_summary_generation():
    """Test AI summary generation with mocking."""
    state = {
        "original_document": "This is a test document for summarization testing.",
        "version_history": []
    }
    
    with patch('langchain_openai.ChatOpenAI') as mock_llm_class:
        mock_llm = MagicMock()
        mock_response = MagicMock()
        mock_response.content = "This is a test summary of the document."
        mock_llm.invoke.return_value = mock_response
        mock_llm_class.return_value = mock_llm
        
        result = generate_ai_summary(state)
        
        assert "ai_summary" in result
        assert result["ai_summary"] == "This is a test summary of the document."
        assert result["processing_stage"] == "summary_generated"
        assert len(result["version_history"]) == 1
        
    print("✅ test_summary_generation passed")

def test_routing_logic():
    """Test routing decisions based on state."""
    # Test approved routing
    approved_state = {
        "summary_approved": True,
        "processing_stage": "summary_approved"
    }
    assert route_after_editing(approved_state) == "follow_up"
    
    # Test regeneration routing
    regen_state = {
        "summary_approved": False,
        "processing_stage": "regenerate_requested"
    }
    assert route_after_editing(regen_state) == "regenerate"
    
    # Test error routing
    error_state = {
        "processing_stage": "error"
    }
    assert route_after_editing(error_state) == "end"
    
    print("✅ test_routing_logic passed")

def test_error_handling():
    """Test error handling in document processing."""
    # Test with invalid document
    invalid_state = {"messages": []}
    result = process_document(invalid_state)
    
    assert "Error" in result["original_document"]
    assert result["processing_stage"] == "error"
    
    # Test summary generation with error
    error_doc_state = {"original_document": "Error: Test error"}
    result = generate_ai_summary(error_doc_state)
    
    assert "Cannot summarize" in result["ai_summary"]
    
    print("✅ test_error_handling passed")

# Run the tests
print("🧪 Running unit tests for document summary workflow...")
test_document_validation()
test_version_history()
test_diff_functionality()
test_summary_generation()
test_routing_logic()
test_error_handling()
print("\n✅ All unit tests passed!")

## Explanation

### Problem Solved
This intermediate example demonstrates a practical human-in-the-loop workflow where:
- AI generates an initial document summary
- Human reviews and can edit the summary for accuracy
- The human-edited summary is used for follow-up tasks
- Version history tracks all changes

### Graph Structure
The workflow follows this enhanced flow:
1. **Document Processing**: Validate and prepare the input document
2. **AI Summary Generation**: Create initial summary using GPT-4
3. **Human Editing Interface**: Present summary for human review/editing
4. **Conditional Flow**: Based on human decision (approve/edit/regenerate)
5. **Follow-up Task**: Use approved summary for additional AI tasks
6. **Completion**: Finalize with results and statistics

### Human Intervention Points
The human can intervene at multiple points:
- **Approval**: Accept the AI summary as-is
- **Editing**: Directly modify the summary text
- **Regeneration**: Request AI to improve based on feedback
- **Follow-up Selection**: Choose what task to perform with the summary

### Graph Adaptation
The graph adapts to human input by:
- **Version Tracking**: Maintaining history of all summary versions
- **Feedback Integration**: Using human feedback to regenerate improved summaries
- **Flexible Routing**: Supporting multiple editing cycles until approval
- **Task Customization**: Allowing human choice of follow-up tasks

### Advanced Features
- **Diff Visualization**: Shows exact changes between AI and human versions
- **Multi-modal Editing**: Support for approval, direct editing, or regeneration
- **Version History**: Complete audit trail of all changes
- **Error Recovery**: Graceful handling of processing errors
- **Flexible Follow-up**: Multiple follow-up task options based on the refined summary

This example demonstrates how human expertise can be seamlessly integrated into AI workflows to improve quality and accuracy of the final output.